# Simple Normalized Coordinates

1D normalized coordinates originate from the normal form decomposition, where the transfer matrix that propagates phase space coordinates $(x, p)$ is decomposed as

$M = A \cdot R(\theta) \cdot A^{-1}$

And the matrix $A$ can be parameterized as

A = $\begin{pmatrix}\sqrt{\beta} & 0\\-\alpha/\sqrt{\beta} & 1/\sqrt{\beta}\end{pmatrix}$


In [ ]:
from pmd_beamphysics import ParticleGroup
from pmd_beamphysics.statistics import (
    A_mat_calc,
    twiss_calc,
    normalized_particle_coordinate,
    twiss_ellipse_points,
)
import numpy as np
import matplotlib.pyplot as plt
import matplotlib

%config InlineBackend.figure_format = 'retina'

In [ ]:
help(A_mat_calc)

Make phase space circle. This will represent some normalized coordinates:

In [ ]:
theta = np.linspace(0, np.pi * 2, 100)
zvec0 = np.array([np.cos(theta), np.sin(theta)])
plt.scatter(*zvec0)

Make a 'beam' in 'lab coordinates':

In [ ]:
MYMAT = np.array([[10, 0], [-3, 5]])
zvec = np.matmul(MYMAT, zvec0)
plt.scatter(*zvec)

With a beam, $\alpha$ and $\beta$ can be determined from moments of the covariance matrix.

In [ ]:
help(twiss_calc)

Calculate a sigma matrix, get the determinant:

In [ ]:
sigma_mat2 = np.cov(*zvec)
np.linalg.det(sigma_mat2)

Get some twiss:

In [ ]:
twiss = twiss_calc(sigma_mat2)
twiss

Analyzing matrices:

In [ ]:
A = A_mat_calc(twiss["beta"], twiss["alpha"])
A_inv = A_mat_calc(twiss["beta"], twiss["alpha"], inverse=True)

A_inv turns this back into a circle:

In [ ]:
zvec2 = np.matmul(A_inv, zvec)
plt.scatter(*zvec2)

# Twiss parameters

Effective Twiss parameters can be calculated from the second order moments of the particles.

This does not change the phase space area.


In [ ]:
twiss_calc(np.cov(*zvec2))

In [ ]:
matplotlib.rcParams["figure.figsize"] = (13, 8)  # Reset plot

# x_bar, px_bar, Jx, etc.

These are essentially action-angle coordinates, calculated by using the an analyzing twiss dict

In [ ]:
help(normalized_particle_coordinate)

Get some example particles, with a typical transverse phase space plot:

In [ ]:
P = ParticleGroup("data/bmad_particles2.h5")
P.plot("x", "px", ellipse=True)

If no twiss is given, then the analyzing matrix is computed from the beam itself:

In [ ]:
normalized_particle_coordinate(P, "x", twiss=None)

This is equivelent:

In [ ]:
normalized_particle_coordinate(
    P, "x", twiss=twiss_calc(P.cov("x", "px")), mass_normalize=False
) / np.sqrt(P.mass)

And is given as a property:

In [ ]:
P.x_bar

The amplitude is defined as:

In [ ]:
(P.x_bar**2 + P.px_bar**2) / 2

This is also given as a property:

In [ ]:
P.Jx

 Note the mass normalization is the same:

In [ ]:
P.Jx.mean(), P["mean_Jx"], P["norm_emit_x"]

This is now nice and roundish:

In [ ]:
P.plot("x_bar", "px_bar")

Jy also works. This gives some sense of where the emittance is larger.

In [ ]:
P.plot("t", "Jy")

Sort by Jx:

In [ ]:
P = P[np.argsort(P.Jx)]

Now particles are ordered:

In [ ]:
plt.plot(P.Jx)

This can be used to calculate the 95% emittance:

In [ ]:
P[0 : int(0.95 * len(P))]["norm_emit_x"]

# Plot ellipse

For convenience this function is available

In [ ]:
x, p = twiss_ellipse_points(sigma_mat2, n_points=100)
plt.plot(x, p, color="red")
plt.scatter(x, p, color="red")

# Simple 'matching'

Often a beam needs to be 'matched' for tracking in some program.

This is a 'faked' tranformation that ultimately would need to be realized by a focusing system.

In [ ]:
def twiss_match(x, p, beta0=1, alpha0=0, beta1=1, alpha1=0):
    """
    Simple Twiss matching.

    Takes positions x and momenta p, and transforms them according to
    initial Twiss parameters:
        beta0, alpha0
    into final  Twiss parameters:
        beta1, alpha1

    This is simply the matrix ransformation:
        xnew  = (   sqrt(beta1/beta0)                  0                 ) . ( x )
        pnew    (  (alpha0-alpha1)/sqrt(beta0*beta1)   sqrt(beta0/beta1) )   ( p )


    Returns new x, p

    """
    m11 = np.sqrt(beta1 / beta0)
    m21 = (alpha0 - alpha1) / np.sqrt(beta0 * beta1)

    xnew = x * m11
    pnew = x * m21 + p / m11

    return xnew, pnew

Get some Twiss:

In [ ]:
T0 = twiss_calc(P.cov("x", "xp"))
T0

 Make a copy and maniplulate:

In [ ]:
P2 = P.copy()
P2.x, P2.px = twiss_match(
    P.x, P.px / P["mean_p"], beta0=T0["beta"], alpha0=T0["alpha"], beta1=9, alpha1=-2
)
P2.px *= P["mean_p"]

In [ ]:
twiss_calc(P2.cov("x", "xp"))

This is a dedicated routine:

In [ ]:
def matched_particles(
    particle_group, beta=None, alpha=None, plane="x", p0c=None, inplace=False
):
    """
    Perfoms simple Twiss 'matching' by applying a linear transformation to
        x, px if plane == 'x', or x, py if plane == 'y'

    Returns a new ParticleGroup

    If inplace, a copy will not be made, and changes will be done in place.

    """

    assert plane in ("x", "y"), f"Invalid plane: {plane}"

    if inplace:
        P = particle_group
    else:
        P = particle_group.copy()

    if not p0c:
        p0c = P["mean_p"]

    # Use Bmad-style coordinates.
    # Get plane.
    if plane == "x":
        x = P.x
        p = P.px / p0c
    else:
        x = P.y
        p = P.py / p0c

    # Get current Twiss
    tx = twiss_calc(np.cov(x, p, aweights=P.weight))

    # If not specified, just fill in the current value.
    if alpha is None:
        alpha = tx["alpha"]
    if beta is None:
        beta = tx["beta"]

    # New coordinates
    xnew, pnew = twiss_match(
        x, p, beta0=tx["beta"], alpha0=tx["alpha"], beta1=beta, alpha1=alpha
    )

    # Set
    if plane == "x":
        P.x = xnew
        P.px = pnew * p0c
    else:
        P.y = xnew
        P.py = pnew * p0c

    return P


# Check
P3 = matched_particles(P, beta=None, alpha=-4, plane="y")
P.twiss(plane="y"), P3.twiss(plane="y")

These functions are in statistics:

In [ ]:
from pmd_beamphysics.statistics import twiss_match, matched_particles